<a href="https://colab.research.google.com/github/XieSandi/DicodingMLCourse-Exercise/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Dataset**

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
import zipfile,os,shutil
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

**Installing Library / Package**

In [ ]:
pip install split-folders

**Importing Library**

In [ ]:
import tensorflow as tf
import split-folder

In [ ]:
#creating new folder
train_path = "/tmp/rockpaperscissors/train"

try:
    os.mkdir(train_path)
except OSError:
    print ("Waduh !")
else:
    print ("Mantap !")


In [ ]:
try:
    #Moving Folder into train
    shutil.move("/tmp/rockpaperscissors/paper", "/tmp/rockpaperscissors/train")
    shutil.move("/tmp/rockpaperscissors/rock", "/tmp/rockpaperscissors/train")
    shutil.move("/tmp/rockpaperscissors/scissors", "/tmp/rockpaperscissors/train")

    #Renaming folder info validation
    os.rename("/tmp/rockpaperscissors/rps-cv-images", "/tmp/rockpaperscissors/validation")
except OSError:
    print ("Waduh !")
else:
    print ("Mantap !")

In [ ]:
base_train_dir = '/tmp/rockpaperscissors/train'
base_validation_dir ='/tmp/rockpaperscissors/validation'

In [ ]:
# training
train_paper_dir = os.path.join(base_train_dir, 'paper')
train_rock_dir = os.path.join(base_train_dir, 'rock')
train_scissor_dir = os.path.join(base_train_dir, 'scissor')
 
#validating
validation_paper_dir = os.path.join(base_validation_dir, 'paper')
validation_rock_dir = os.path.join(base_validation_dir, 'rock')
validation_scissor_dir = os.path.join(base_validation_dir, 'scissor')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        base_train_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        base_validation_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile model with 'adam' optimizer loss function 'binary_crossentropy' 
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

# latih model with model.fit 
model.fit(
      train_generator,
      steps_per_epoch=5,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=5,
      verbose=2)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  
  #String output template
  paper     = "[[1. 0. 0.]]"
  rock      = "[[0. 1. 0.]]"
  scissors  = "[[0. 0. 1.]]"
  
  #convert numpy array into string
  result = np.array_str(classes) 
  
  
  print('\n')
  print('Predict RESULT')
  print(fn)
  print(result)


  #Changing result into human language
  if result == paper :
     print("paper")
  elif result == rock :
     print("rock")
  elif result == scissors :
     print("scissors")
  else :
     print("[[%PAPER. %ROCK. %SCISSORS]]")